In [1]:
%%capture
!pip install aequitas

In [2]:
import numpy as np

import pandas as pd
from aequitas.group import Group

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

# Tarea 1: Pipeline Aequitas

El objetivo de esta tarea es evaluar el uso del toolkit Aequitas para obtener métricas de grupo. En particular, trabajarán con los datos de riesgo crediticio alemán que ya fueron utilizados en el Lab Formativo de la semana 1, en el cual debían realizar el análisis exploratorio de estos datos. 

En este notebook se implementa, a partir de estos datos, un modelo simple de Regresión Logística el cual es utilizado para predecir el riesgo crediticio (bueno/malo), por lo tanto ustedes **NO DEBEN** implementar un modelo de clasificación, sino evaluar las métricas de grupo del modelo que se les entrega.

Esta tarea es **individual**. 

In [86]:
# En esta celda de código se desarrolla el modelo de Regresión Logística.

df = pd.read_csv('german_risk.csv')
df1 = df.copy()

# Binarización de algunas variables.

df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})

# Separamos X e Y.

X = df.loc[:, df.columns != 'credit-risk']
y = df.loc[:, df.columns == 'credit-risk']

# Obtenemos variables dummies.

catcols = X.select_dtypes(exclude='number').columns
ignore = ['sex', 'age_cat', 'foreign_worker']
for catcol in catcols:
    if catcol in ignore:
        pass
    else:
        dummies = pd.get_dummies(X[catcol])
        X = pd.concat([X, dummies], axis=1).drop(columns=[catcol])


# Datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)

# No utilizaremos la edad

X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)

# Regresión logística

lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)
lr.fit(X_train, np.ravel(y_train))

# Predicciones del modelo

y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

# DataFrame Final

to_eval = X_test.copy()
to_eval['score'] = y_pred
to_eval['label'] = y_test

In [87]:
to_eval.head()

,sex,age_cat,foreign_worker,duration,credit_amount,installment_commitment,residence_since,existing_credits,num_dependents,0<=X<200,...,unemp/unskilled non res,unskilled resident,none,yes,div/dep/mar,div/sep,mar/wid,single,score,label
298,1,1,0,18.0,2515.0,3.0,4.0,1.0,1.0,0,...,0,0,0,1,0,0,0,1,True,1
273,1,1,0,48.0,3060.0,4.0,4.0,2.0,1.0,1,...,0,0,1,0,0,0,0,1,True,0
143,1,0,0,18.0,2462.0,2.0,2.0,1.0,1.0,0,...,0,0,1,0,0,0,0,1,True,0
59,0,0,0,36.0,6229.0,4.0,4.0,2.0,1.0,0,...,0,1,0,1,1,0,0,0,False,0
154,1,1,0,24.0,6967.0,4.0,4.0,1.0,1.0,1,...,0,0,0,1,0,0,0,1,True,1


## 1. Prepara los datos `to_eval` para poder evaluarlos con Aequitas.

Para ello, define la función
```python
def pre_processing(to_eval):
    ...
    return df
```
que recibe los datos y los pre-procesa para utilizarlos en Aequitas. La función debe retornar el DataFrame obtenido. Puedes encontrar más detalles en la [documentación de Aequitas](https://dssg.github.io/aequitas/input_data.html#Input-data-for-Python-package). Asegúrese de dejar las variables binarias en formato numérico y no booleano. Utiliza los atributos `sex`, `age_cat` y `foreign_worker`.

In [88]:
def pre_processing(to_eval):
    
    df2 = to_eval.copy()
    
    
    df2['sex'] = df2['sex'].map({1: 'male', 0: 'female'})
    df2['age_cat'] = df2['age_cat'].map({1: 'aged', 0: 'young'})
    df2['foreign_worker'] = df2['foreign_worker'].map({1: 'no', 0: 'yes'})
    
    
    df2['score'] = df2['score'].map({True: 1, False: 0})
    
    
    df2.rename(columns={'label': 'label_value'}, inplace=True)
    
    
    cat_cols = ['sex', 'age_cat', 'foreign_worker']
    for col in cat_cols:
        df2[col] = df2[col].astype(str)
    
    
    df = df2[['score', 'label_value', 'foreign_worker', 'age_cat', 'sex']]
    
    return df

    
    

In [89]:
assert pre_processing(to_eval)['sex'].dtype == 'object'
assert 'male' in pre_processing(to_eval)['sex'].unique()
assert pre_processing(to_eval)['score'].dtype in ['int', 'float']

## 2. Calcula las métricas grupales con Aequitas.

Para ello, utiliza la clase `Group()` de Aequitas. Define la función
```python
def group_metrics(to_eval):
    ...
    return df
```
que recibe el DataFrame ya pre-procesado en la pregunta anterior y retorna el DataFrame de las métricas grupales (fpr, fnr, fdr, for, etc).

In [92]:
def group_metrics(to_eval):
       
    bias = Group()
    df, _ = bias.get_crosstabs(to_eval)
    
    return df
    

In [93]:
# Tests
female = group_metrics(pre_processing(to_eval)).query("attribute_value == 'female'").reset_index(drop=True)
male = group_metrics(pre_processing(to_eval)).query("attribute_value == 'male'").reset_index(drop=True)

assert (female['fp'] / female['group_label_neg'])[0] == female['fpr'][0]
assert (male['fp'] / male['group_label_neg'])[0] == male['fpr'][0]